In [1]:
!nvidia-smi

Thu May  2 10:37:12 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN Xp            Off  | 00000000:01:00.0 Off |                  N/A |
| 23%   26C    P8    16W / 250W |      0MiB / 12195MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
IMG_WIDTH = 1400
IMG_HEIGHT = 1400
ROW_SLICE = slice(0, 1400)
COL_SLICE = slice(1000, None)

In [3]:
import os
import sys
import numpy as np
sys.path.append("../")
from matplotlib.image import imread
import matplotlib.pyplot as plt
import tensorflow as tf
from UNet import get_unet

Using TensorFlow backend.


In [4]:
%matplotlib inline

In [6]:
data_dir = "/storage/yw18581/data/"

In [ ]:
from data_loaders_km3 import data_generator, get_n_iterations
from os import path as p
from tqdm import tqdm

In [ ]:
def inference_step(network_model, test_data_generator, predict_steps):

    y_pred = list()

    for _ in tqdm(range(predict_steps)):
        X_batch, _ = next(test_data_generator)
        Y_batch_pred = network_model.predict_on_batch(X_batch)
        #print(Y_batch_pred.shape)
        y_pred.append(Y_batch_pred)
    y_pred = np.concatenate(y_pred, axis=0)

    return y_pred

In [ ]:
TRAIN_VAL_TEST_DIR = os.path.join(data_dir, "train_validation_test")
N_EPOCHS=1500
N_FILES = 1
BATCH_SIZE=2

In [ ]:
CHECKPOINT_FOLDER_PATH = os.path.join(data_dir, 'trained_models')
TASK_NAME = 'UNet_training_generator_{}epochs'.format(N_EPOCHS)
TASK_FOLDER_PATH = os.path.join(CHECKPOINT_FOLDER_PATH, TASK_NAME)

In [ ]:
TRAINING_WEIGHTS_FILEPATH=os.path.join(CHECKPOINT_FOLDER_PATH,'retrained_UNet_1000+500epochs.hdf5')

In [ ]:
model1500 = get_unet()
model1500.load_weights(TRAINING_WEIGHTS_FILEPATH)

In [13]:
fname_1mm = [os.path.join(TRAIN_VAL_TEST_DIR,"Xy_1mm.npz")]
fname_3mm = [os.path.join(TRAIN_VAL_TEST_DIR,"Xy_3mm.npz")]
fname_15mm = [os.path.join(TRAIN_VAL_TEST_DIR,"Xy_15mm.npz")]
fname_20mm = [os.path.join(TRAIN_VAL_TEST_DIR,"Xy_20mm.npz")]
fname_30mm = [os.path.join(TRAIN_VAL_TEST_DIR,"Xy_30mm.npz")]
fname_35mm = [os.path.join(TRAIN_VAL_TEST_DIR,"Xy_35mm.npz")]

In [14]:
fname_2mm = [os.path.join(TRAIN_VAL_TEST_DIR,"Xy_2mm.npz")]
fname_4mm = [os.path.join(TRAIN_VAL_TEST_DIR,"Xy_4mm.npz")]
fname_10mm = [os.path.join(TRAIN_VAL_TEST_DIR,"Xy_10mm.npz")]
fname_25mm = [os.path.join(TRAIN_VAL_TEST_DIR,"Xy_25mm.npz")]

In [15]:
def data_gen(fname):
    return data_generator(fname, batch_size=BATCH_SIZE, ftarget = lambda y: y)

In [16]:
data_gen_1mm = data_gen(fname_1mm)
data_gen_3mm = data_gen(fname_3mm)
data_gen_15mm = data_gen(fname_15mm)
data_gen_20mm = data_gen(fname_20mm)
data_gen_30mm = data_gen(fname_30mm)
data_gen_35mm = data_gen(fname_35mm)
data_gen_2mm = data_gen(fname_2mm)
data_gen_4mm = data_gen(fname_4mm)
data_gen_10mm = data_gen(fname_10mm)
data_gen_25mm = data_gen(fname_25mm)

In [17]:
for i in [fname_3mm, fname_15mm, fname_20mm, fname_30mm, fname_35mm, fname_2mm, fname_4mm, fname_10mm, fname_25mm]:
    steps, evts = get_n_iterations(i, batch_size=BATCH_SIZE)
    print(steps, evts)

['/storage/yw18581/data/train_validation_test/Xy_3mm.npz']
/storage/yw18581/data/train_validation_test/Xy_3mm.npz
(300, 1400, 1400, 1)
150 300
['/storage/yw18581/data/train_validation_test/Xy_15mm.npz']
/storage/yw18581/data/train_validation_test/Xy_15mm.npz
(300, 1400, 1400, 1)
150 300
['/storage/yw18581/data/train_validation_test/Xy_20mm.npz']
/storage/yw18581/data/train_validation_test/Xy_20mm.npz
(300, 1400, 1400, 1)
150 300
['/storage/yw18581/data/train_validation_test/Xy_30mm.npz']
/storage/yw18581/data/train_validation_test/Xy_30mm.npz
(300, 1400, 1400, 1)
150 300
['/storage/yw18581/data/train_validation_test/Xy_35mm.npz']
/storage/yw18581/data/train_validation_test/Xy_35mm.npz
(300, 1400, 1400, 1)
150 300
['/storage/yw18581/data/train_validation_test/Xy_2mm.npz']
/storage/yw18581/data/train_validation_test/Xy_2mm.npz
(300, 1400, 1400, 1)
150 300
['/storage/yw18581/data/train_validation_test/Xy_4mm.npz']
/storage/yw18581/data/train_validation_test/Xy_4mm.npz
(300, 1400, 1400, 1)

In [18]:
prediction_steps, n_evts = steps, evts

In [19]:
def run_predictions_and_save(model, fname_in, data_gen, steps, file_key):
    pred = inference_step(model, data_gen, steps)
    pred = pred.astype(np.uint8)*255
    Xd = np.load(fname_in[0])
    X_out =Xd['x'][...,0]
    dist_out = Xd['dist']
    np.savez_compressed(os.path.join(TASK_FOLDER_PATH,"Xy_"+file_key+"_predicted_UNet.npz"),
                   x= X_out, y=pred[...,0], dist = dist_out)
    return

In [20]:
run_predictions_and_save(model1500, fname_1mm, data_gen_1mm, prediction_steps, "1mm")

100%|██████████| 150/150 [00:40<00:00,  4.95it/s]


In [23]:
os.listdir(TASK_FOLDER_PATH)

['Xy_test_predicted_UNet.npz',
 'Xy_3mm_predicted_UNet.npz',
 'Xy_1mm_predicted_UNet.npz']

In [25]:
run_predictions_and_save(model1500, fname_2mm, data_gen_2mm, prediction_steps, "2mm")

100%|██████████| 150/150 [00:39<00:00,  4.90it/s]


In [26]:
run_predictions_and_save(model1500, fname_4mm, data_gen_4mm, prediction_steps, "4mm")

100%|██████████| 150/150 [00:39<00:00,  4.88it/s]


In [27]:
run_predictions_and_save(model1500, fname_10mm, data_gen_10mm, prediction_steps, "10mm")

100%|██████████| 150/150 [00:40<00:00,  4.91it/s]


In [28]:
run_predictions_and_save(model1500, fname_15mm, data_gen_15mm, prediction_steps, "15mm")

100%|██████████| 150/150 [00:40<00:00,  4.86it/s]


In [19]:
run_predictions_and_save(model1500, fname_20mm, data_gen_20mm, prediction_steps, "20mm")

100%|██████████| 150/150 [00:44<00:00,  4.89it/s]


In [20]:
run_predictions_and_save(model1500, fname_25mm, data_gen_25mm, prediction_steps, "25mm")

100%|██████████| 150/150 [00:38<00:00,  4.93it/s]


In [20]:
run_predictions_and_save(model1500, fname_30mm, data_gen_30mm, prediction_steps, "30mm")

100%|██████████| 150/150 [01:16<00:00,  4.90it/s]


In [21]:
run_predictions_and_save(model1500, fname_35mm, data_gen_35mm, prediction_steps, "35mm")

100%|██████████| 150/150 [00:38<00:00,  4.91it/s]
